In [1]:
!pip install --upgrade google-generativeai

In [2]:
import os
from getpass import getpass

# Secure API key input
os.environ["GOOGLE_API_KEY"] = getpass("🔑 Enter your Google API Key: ")

🔑 Enter your Google API Key: ··········


In [3]:
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Load Gemini 1.5 Flash
model = genai.GenerativeModel("models/gemini-1.5-flash-latest")

In [8]:
import re
import json

def classify_hazard(text):
    prompt = f"""
You are a certified workplace safety AI. Your job is to analyze hazard reports and classify them into severity levels.

There are 3 severity levels:
1️⃣ High (severity = 1): Immediate danger, injury likely or already occurred, requires urgent action.
2️⃣ Medium (severity = 2): Moderate hazard, no injury yet, but risk exists and action is needed soon.
3️⃣ Low (severity = 3): Minor issue, unlikely to cause harm, monitor or fix when convenient.

--- Examples ---

Example 1:
Input: "Oil spill near power unit. Workers slipping. Machines still running."
Output:
{{
  "hazard_level": "High",
  "severity": 1,
  "report_text": "..."
}}

Example 2:
Input: "Forklift missing reverse beeper in shared aisle. No injury yet."
Output:
{{
  "hazard_level": "Medium",
  "severity": 2,
  "report_text": "..."
}}

Example 3:
Input: "Coffee spill near microwave. Cleaned up. No one slipped."
Output:
{{
  "hazard_level": "Low",
  "severity": 3,
  "report_text": "..."
}}

--- Classify This ---

Input: "{text}"

Respond only with a JSON object with the following keys:
- hazard_level (string: High, Medium, or Low)
- severity (integer: 1, 2, or 3)
- report_text (string: full formatted safety report)
"""
    response = model.generate_content(prompt)
    raw_output = response.text

    # Extract JSON block from response (even if wrapped in code)
    match = re.search(r"\{.*\}", raw_output, re.DOTALL)
    if match:
        try:
            return json.loads(match.group())
        except Exception as e:
            raise ValueError("❌ Gemini response was not valid JSON. Fix prompt or parse manually.") from e
    else:
        raise ValueError("❌ No JSON found in Gemini response.")

In [5]:
from datetime import datetime

def run_alertaid(input_text):
    print("🧠 Classifying hazard...")
    hazard = classify_hazard(input_text)

    print("📊 Classification Result:")
    print(json.dumps(hazard, indent=2))

    print("\n📄 Full Safety Report:\n")
    print(hazard["report_text"])  # Always print the report

    severity = hazard.get("severity", 3)

    if severity == 1:
        subject = "🚨 High-Risk Workplace Hazard Alert"
        recipient = "receiveremail@gmail.com"
        body = hazard["report_text"]
        send_email(subject, body, recipient)

    elif severity == 2:
        log_entry = f"\n==== MEDIUM-RISK HAZARD ====\nTimestamp: {datetime.now()}\n{hazard['report_text']}\n"
        with open("medium_risk_log.txt", "a") as log_file:
            log_file.write(log_entry)
        print("📝 Medium-risk hazard logged to 'medium_risk_log.txt'.")

    else:
        print("✅ Low-risk hazard. No action required.")
        print("🔍 Recommendation from Gemini:")
        report = hazard["report_text"]
        if "Recommendations:" in report:
            print(report.split("Recommendations:")[-1].strip())
        else:
            print("No specific recommendations found.")

In [6]:
import smtplib
import ssl
from email.message import EmailMessage
import getpass

def send_email(subject, body, recipient_email):
    sender_email = "nityachintan@gmail.com"
    app_password = getpass.getpass("Enter your Gmail App Password: ")

    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg.set_content(body)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(sender_email, app_password)
        smtp.send_message(msg)

    print("✅ Email sent successfully!")


In [13]:
# High Risk, sevierity 1
run_alertaid("Oil spill near main power unit. No signage. Workers slipping and nearby machines are active.")

🧠 Classifying hazard...
🔍 Gemini Raw Output:
 ```json
{
  "hazard_level": "High",
  "severity": 1,
  "report_text": "Safety Incident Report\n\nDate: October 26, 2023\nTime: 10:45 AM\nLocation: Near main power unit\n\nDescription: An oil spill occurred near the main power unit, creating a slippery surface.  No warning signage was present. Workers reported near misses due to slipping, and nearby machinery remained active, increasing the risk of serious injury.\n\nInjuries Reported: No injuries reported, but several near misses due to slipping.\n\nWitnesses: Ravi Patel (Technician), Aisha Khan (Supervisor)\n\nActions Taken: Area cordoned off; oil spill cleanup initiated; warning signs posted; machinery in the vicinity shut down.\n\nRecommendations: Implement a preventative maintenance program for power unit; improve spill response procedures; provide additional safety training for all workers; install improved oil containment systems."
}
```
📊 Classification Result:
{
  "hazard_level": "H

In [9]:
# Medium Risk, sevierity 2
run_alertaid("Loose cable running across the hallway in the packing section. No immediate injuries, but a worker tripped and almost fell. The area is usually crowded during shift changes.")

🧠 Classifying hazard...
📊 Classification Result:
{
  "hazard_level": "Medium",
  "severity": 2,
  "report_text": "Loose cable running across the hallway in the packing section. No immediate injuries, but a worker tripped and almost fell. The area is usually crowded during shift changes."
}

📄 Full Safety Report:

Loose cable running across the hallway in the packing section. No immediate injuries, but a worker tripped and almost fell. The area is usually crowded during shift changes.
📝 Medium-risk hazard logged to 'medium_risk_log.txt'.


In [12]:
# Low Risk, sevierity 3
run_alertaid("A small puddle of water was found near the water cooler due to condensation. It was wiped up promptly. No one slipped.")

🧠 Classifying hazard...
📊 Classification Result:
{
  "hazard_level": "Low",
  "severity": 3,
  "report_text": "A small puddle of water was found near the water cooler due to condensation. It was wiped up promptly. No one slipped."
}

📄 Full Safety Report:

A small puddle of water was found near the water cooler due to condensation. It was wiped up promptly. No one slipped.
✅ Low-risk hazard. No action required.
🔍 Recommendation from Gemini:
No specific recommendations found.
